# The Effect of Proximity To Top-tier Universities On Startups
### by Jessica Dowuona-Owoo

##### Introduction

In [24]:
import matplotlib
import matplotlib.colors as mplc
import matplotlib.patches as patches
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [67]:
objects = pd.read_csv('/Users/jessyterlisner/Desktop/ECO225Project/Data/objects.csv', low_memory=False)
investments = pd.read_csv('/Users/jessyterlisner/Desktop/ECO225Project/Data/investments.csv')
office = pd.read_csv('/Users/jessyterlisner/Desktop/ECO225Project/Data/offices.csv')
milestones = pd.read_csv('/Users/jessyterlisner/Desktop/ECO225Project/Data/milestones.csv')
funds = pd.read_csv('/Users/jessyterlisner/Desktop/ECO225Project/Data/funds.csv')
ipos = pd.read_csv('/Users/jessyterlisner/Desktop/ECO225Project/Data/ipos.csv')
f_rounds = pd.read.csv('/Users/jessyterlisner/Desktop/ECO225Project/Data/funding_rounds.csv')

# renaming values
objects.rename(columns={'id':'funded_object_id'}, inplace=True)


#remove unwanted columns
obj = objects.drop(columns= ['created_at', 'updated_at', 'tag_list', 'logo_height', 'logo_width', 'logo_url', 'created_by', 'twitter_username', 'homepage_url'], inplace= False)
inv = investments.drop(columns= ['created_at', 'updated_at'], inplace=False)


In [73]:
source = pd.merge(obj, inv, on= 'funded_object_id')
source.head()

,id_x,entity_type,entity_id,parent_id,name,normalized_name,permalink,category_code,status,founded_at,...,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,id_y,funding_round_id,funded_object_id,investor_object_id
0,c:1,Company,1,NaN,Wetpaint,wetpaint,/company/wetpaint,web,operating,2005-10-17,...,3,39750000.0,2010-09-05,2013-09-18,5,17,1289,888,c:1,f:430
1,c:1,Company,1,NaN,Wetpaint,wetpaint,/company/wetpaint,web,operating,2005-10-17,...,3,39750000.0,2010-09-05,2013-09-18,5,17,1290,888,c:1,f:3
2,c:1,Company,1,NaN,Wetpaint,wetpaint,/company/wetpaint,web,operating,2005-10-17,...,3,39750000.0,2010-09-05,2013-09-18,5,17,1291,889,c:1,f:4
3,c:1,Company,1,NaN,Wetpaint,wetpaint,/company/wetpaint,web,operating,2005-10-17,...,3,39750000.0,2010-09-05,2013-09-18,5,17,1292,889,c:1,f:430
4,c:1,Company,1,NaN,Wetpaint,wetpaint,/company/wetpaint,web,operating,2005-10-17,...,3,39750000.0,2010-09-05,2013-09-18,5,17,1293,889,c:1,f:3


#### Summary Statistics

#### Data Visualization


In [ ]:
total_funding = 
rev_growth = 
startup_val = f_rounds['']

f_rounds = f_rounds[f_rounds['post_money_valuation_usd'].notna()]
f_rounds = f_rounds[f_rounds['post_money_valuation_usd'] > 0]

